In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [8]:
df=pd.read_csv("covid_symptoms.csv")
df

,age,gender,vaccination_status,fever,cough,fatigue,shortness_of_breath,loss_of_smell,headache,diabetes,hypertension,heart_disease,asthma,cancer,hospitalized,icu_admission,mortality
0,51,Male,Unvaccinated,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,92,Male,Fully Vaccinated,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,14,Male,Fully Vaccinated,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,71,Male,Fully Vaccinated,1,1,0,0,0,0,0,1,0,0,0,0,0,0
4,60,Male,Unvaccinated,0,1,1,1,1,0,1,1,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,23,Male,Fully Vaccinated,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2996,98,Female,Fully Vaccinated,1,0,0,0,1,0,1,0,1,1,0,1,0,0
2997,25,Male,Partially Vaccinated,0,0,0,0,1,0,1,0,0,0,1,0,0,0
2998,36,Female,Fully Vaccinated,1,0,0,1,0,1,0,0,0,0,0,0,0,0


In [9]:
df.columns

Index(['age', 'gender', 'vaccination_status', 'fever', 'cough', 'fatigue',
       'shortness_of_breath', 'loss_of_smell', 'headache', 'diabetes',
       'hypertension', 'heart_disease', 'asthma', 'cancer', 'hospitalized',
       'icu_admission', 'mortality'],
      dtype='object')

In [10]:
df["covid"]=(
    (df["fever"]==1)&
    (df["cough"]==1)&
    (df["shortness_of_breath"]==1)&
    (df["headache"]==1)
).astype(int)

In [12]:
cat_cols=["gender","vaccination_status"]
le=LabelEncoder()

for col in cat_cols:
    df[col]=le.fit_transform(df[col].astype(str))

In [13]:
# Feature & Target values
X=df[["fever","cough","shortness_of_breath","headache"]]
y=df["covid"]


In [14]:
# Train-Test-Split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [17]:
model=lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)
model.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 19, number of negative: 2081
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000251 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8
[LightGBM] [Info] Number of data points in the train set: 2100, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.009048 -> initscore=-4.696165
[LightGBM] [Info] Start training from score -4.696165
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,3
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [18]:
# Prediction
y_pred=model.predict(X_test)

In [19]:
print("Accuracy Score:",accuracy_score(y_test,y_pred))

Accuracy Score: 1.0


In [20]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       890
           1       1.00      1.00      1.00        10

    accuracy                           1.00       900
   macro avg       1.00      1.00      1.00       900
weighted avg       1.00      1.00      1.00       900



In [23]:
# fever, cough, fatigue, breathlessness, sore_throat, headache
new_patient = [[1, 1, 1, 1]]

prediction = model.predict(new_patient)

if prediction[0] == 1:
    print("⚠️ HIGH COVID RISK – Further testing advised")
else:
    print("✅ LOW COVID RISK")


⚠️ HIGH COVID RISK – Further testing advised


C:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
